In [15]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
import glob, re, os
import subprocess

from smt.applications.mixed_integer import (
    MixedIntegerContext,
    FLOAT,
    ENUM,
    ORD,
)

import scipy.linalg as lng

from smt.applications import EGO
from smt.surrogate_models import KRG
from smt.sampling_methods import LHS


import scienceplots
plt.style.use('default')
plt.style.use(['science', 'high-vis', 'grid'])

In [16]:
PATH_BUILD_DIR = "/work/utox/users/helleboid_work_utox/OptiDopi/build/"
PATH_COST_CPP = "/work/utox/users/helleboid_work_utox/OptiDopi/build/apps/spad_cost_function"
RES_FILE = "RES/res_cost_func.csv"
os.makedirs("EGO_Results/", exist_ok=True)
# Recompile the code first.
#subprocess.run(["make", f"-j 32"], cwd=PATH_BUILD_DIR)

In [17]:
def function_test_1d(length_intrinsic):
    doping_donor = 20.0
    length_intrinsic_donor = 20.0
    subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{doping_acceptor}", RES_FILE])
    length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
    print(f)
    return total_cost

def function_test_2d(PARAMS):
    #print(PARAMS)
    Y = []
    for idx in range(len(PARAMS)):
        length_intrinsic = PARAMS[idx,0]
        #print(f'{length_intrinsic=}')
        log_doping_acceptor =  PARAMS[idx,1]
        subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{log_doping_acceptor}", RES_FILE])
        length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
        Y.append(total_cost)
        print(f"Total cost: {total_cost}")
    return np.array(Y)

vfunc = np.vectorize(function_test_1d)

vfunc2d = np.vectorize(function_test_2d)


In [18]:
min_length_intrinsic = 0.0
max_length_intrinsic = 1.0
min_log_acceptor = 16.5
max_log_acceptor = 19.0


n_iter = 10000
xtypes = [FLOAT, FLOAT]
#mixint = MixedIntegerContext(xtypes, xlimits)
xlimits = np.array([[min_length_intrinsic, max_length_intrinsic], [min_log_acceptor, max_log_acceptor]])

qEI = "KB"
sm = KRG(print_global=False)
mixint = MixedIntegerContext(xtypes, xlimits)
n_doe = 5
sampling = LHS(xlimits=xlimits)
xdoe = sampling(n_doe)
ydoe = function_test_2d(xdoe)



Total cost: nan
Total cost: nan
Total cost: nan
Total cost: nan
Total cost: nan


In [7]:
criterion = "SBO"  #'EI' or 'SBO' or 'LCB'
ego = EGO(n_iter=n_iter, criterion=criterion, xdoe=xdoe, xlimits=xlimits, xtypes=xtypes, qEI=qEI,
          enable_tunneling=True)
x_opt, y_opt, _, x_data, y_data = ego.optimize(fun=function_test_2d)

Total cost: 934355.7
Total cost: 745223.7
Total cost: 24979.29
Total cost: 141875.0
Total cost: 952941.0
Total cost: 7253.857
Total cost: 273.4142
Total cost: 19034990.0
Total cost: 7253.823
Total cost: -7.038889
Total cost: 10000000000.0
Total cost: 250542.8
Total cost: 101045.1
Total cost: 7253.833
Total cost: 57.76083
Total cost: 934356.0
Total cost: -7.04338
Total cost: 72852.25
Total cost: 4584.147
Total cost: 273.4678
Total cost: -6.279182
Total cost: 36270.54
Total cost: 101046.1
Total cost: 934353.0
Total cost: 187213.3
Total cost: 11193.96
Total cost: 138360.4
Total cost: 2816.233
Total cost: 435448.0
Total cost: 435448.0
Total cost: 435448.0
Total cost: 435448.0
Total cost: 435448.0
Total cost: 435449.4
Total cost: 72850.66
Total cost: 4584.316
Total cost: 20.29981
Total cost: -7.190387
Total cost: 187216.2
Total cost: 934341.7
Total cost: 103708.7
Total cost: 103708.7
Total cost: nan
Optimization failed. Try increasing the ``nugget``


ValueError: array must not contain infs or NaNs

In [ ]:
print("Minimum in x={} with f(x)={:.1f}".format(x_opt, float(y_opt)))
print("Minimum in typed x={}".format(ego.mixint.cast_to_mixed_integer(x_opt)))

min_ref = -15
mini = np.zeros(n_iter)
for k in range(n_iter):
    mini[k] = np.log(np.abs(np.min(y_data[0 : k + n_doe - 1]) - min_ref))
x_plot = np.linspace(1, n_iter + 0.5, n_iter)
u = max(np.floor(max(mini)) + 1, -100)
l = max(np.floor(min(mini)) - 0.2, -10)
fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
axes.plot(x_plot, mini, color="r")
axes.set_ylim([l, u])
plt.title("minimum convergence plot", loc="center")
plt.xlabel("number of iterations")
plt.ylabel("log of the difference w.r.t the best")
plt.show()